In [ ]:
print('test : ga ada di jira')

In [1]:
import pandas as pd
import os
import numpy as np
import datetime as dt
from datetime import date, time, datetime
import math
import string

In [2]:
import pandas.io.sql as sqlio
import psycopg2 as ps
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np

In [3]:
load_dotenv()
db = os.getenv('db')
user = os.getenv('user')
password = os.getenv('POSTGRES_PASSWORD')
host = os.getenv('host')
port = os.getenv('port')

In [4]:
conn = ps.connect(dbname=db, 
                  user=user,
                  password=password, 
                  host=host,
                  port=port)

In [5]:
# Accommodate query
sql = """SELECT id_port, port_name
FROM stg_pelni.master_port
where flag_ppss like 'Y'"""

In [6]:
df2 = pd.read_csv('data/min_max_mean.csv')

In [7]:
# Read Query
df1 = sqlio.read_sql_query(sql, conn)

C:\Users\BN001602001\AppData\Local\Temp\ipykernel_7812\2371418704.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df1 = sqlio.read_sql_query(sql, conn)


In [8]:
df2

,Unnamed: 0,id_port,code,ke,min,max,mean
0,1,00739854-654f-469c-b5af-31680f7ae36e,662,Labuan Bajo,1,6,2
1,2,ddac0441-65ff-4960-b993-87c73ebc2d85,322,Belinyu,1,3,1
2,3,0f64cb53-2e97-4fe3-87a0-7b2e77903410,336,Tanjung Pandan,1,6,2
3,4,b2674a83-0456-4666-893d-7c7522f9232b,431,Tanjung Priok,4,18,10
4,5,bd5d57a7-3fec-4c5f-97b2-5d9c448625e0,809,Nunukan/Tunon Taka,1,28,4
...,...,...,...,...,...,...,...
70,71,6a01e7c5-05e6-4ab7-a706-f0936f32a837,893,Makassar,1,19,5
71,72,123da461-dbcb-4de3-9a2a-3575379a4d14,159,Sibolga,4,4,4
72,73,72346087-6a6a-462b-a434-913d6f2e5ca9,942,Ternate,1,8,3
73,74,1334ec95-cc8a-4464-a263-0b0beee559da,251,Sei Kolak Kijang,1,12,3


In [9]:
df1

,id_port,port_name
0,c672024c-821d-4c32-b162-612017b50b4c,Bawean
1,6a52fe82-7596-4930-9105-ad1d85049992,Kahakitang
2,00739854-654f-469c-b5af-31680f7ae36e,Labuan Bajo
3,ddac0441-65ff-4960-b993-87c73ebc2d85,Belinyu
4,0f64cb53-2e97-4fe3-87a0-7b2e77903410,Tanjung Pandan
...,...,...
98,123da461-dbcb-4de3-9a2a-3575379a4d14,Sibolga
99,72346087-6a6a-462b-a434-913d6f2e5ca9,Ternate
100,48e4149c-db11-4979-af3b-c45ee6b94a51,Pomako
101,1334ec95-cc8a-4464-a263-0b0beee559da,Sei Kolak Kijang


In [10]:
df2 = df2.rename(columns={'ke' : 'port_name'})

In [11]:
df_all = df1.merge(df2.drop_duplicates(), on=['id_port','port_name'], 
                   how='left', indicator=True)

In [12]:
df_all

,id_port,port_name,Unnamed: 0,code,min,max,mean,_merge
0,c672024c-821d-4c32-b162-612017b50b4c,Bawean,NaN,NaN,NaN,NaN,NaN,left_only
1,6a52fe82-7596-4930-9105-ad1d85049992,Kahakitang,NaN,NaN,NaN,NaN,NaN,left_only
2,00739854-654f-469c-b5af-31680f7ae36e,Labuan Bajo,1.0,662.0,1.0,6.0,2.0,both
3,ddac0441-65ff-4960-b993-87c73ebc2d85,Belinyu,2.0,322.0,1.0,3.0,1.0,both
4,0f64cb53-2e97-4fe3-87a0-7b2e77903410,Tanjung Pandan,3.0,336.0,1.0,6.0,2.0,both
...,...,...,...,...,...,...,...,...
98,123da461-dbcb-4de3-9a2a-3575379a4d14,Sibolga,72.0,159.0,4.0,4.0,4.0,both
99,72346087-6a6a-462b-a434-913d6f2e5ca9,Ternate,73.0,942.0,1.0,8.0,3.0,both
100,48e4149c-db11-4979-af3b-c45ee6b94a51,Pomako,NaN,NaN,NaN,NaN,NaN,left_only
101,1334ec95-cc8a-4464-a263-0b0beee559da,Sei Kolak Kijang,74.0,251.0,1.0,12.0,3.0,both


In [13]:
cleandf = df_all.loc[df_all['_merge'] == 'left_only', "port_name"]

In [14]:
cleandf

0                   Bawean
1               Kahakitang
15                 Amurang
16                  Amahai
17                Karatung
28              Kolonedale
29               Toli-toli
30                 Banggai
31                  Babang
36                Patimban
37                    Poso
38             Kendawangan
39           Wakai, Togian
40                 Miangas
41                  Tahuna
42                  Lirung
44                   Larat
65                  Belang
66                   Sadai
67               Marapokot
69           Meranti/Dorak
73     Sukabangun Ketapang
74           Pangkalan Bun
76             Selat Lampa
79                     Reo
82                Kotabaru
94                  Tobelo
100                 Pomako
Name: port_name, dtype: object

In [ ]:
df_all['_merge'] == 'left_only'

In [ ]:
common = df1.merge(df2,on=['id_port','port_name'])
(~df1.id_port.isin(common.id_port))&(~df1.port_name.isin(common.port_name))